입체 예시로 철로 된 정육면체에 힘을 가하고, 변위를 예측하여 시각화하는 코드를 제시하겠습니다. 여기서는 3D 유한요소해석(FEA)을 적용하고, 변위를 시각화하기 위해 Python의 NumPy와 Matplotlib을 사용하겠습니다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pyvista as pv

In [2]:
# 정육면체의 크기와 분할할 개수 설정
length = 1.0
num_elements_per_side = 2
num_nodes_per_side = num_elements_per_side + 1

# 각 노드의 위치 생성
nodes = np.linspace(0, length, num_nodes_per_side)
xx, yy, zz = np.meshgrid(nodes, nodes, nodes)
nodes = np.vstack([xx.ravel(), yy.ravel(), zz.ravel()]).T
nodes

array([[0. , 0. , 0. ],
       [0. , 0. , 0.5],
       [0. , 0. , 1. ],
       [0.5, 0. , 0. ],
       [0.5, 0. , 0.5],
       [0.5, 0. , 1. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0.5],
       [1. , 0. , 1. ],
       [0. , 0.5, 0. ],
       [0. , 0.5, 0.5],
       [0. , 0.5, 1. ],
       [0.5, 0.5, 0. ],
       [0.5, 0.5, 0.5],
       [0.5, 0.5, 1. ],
       [1. , 0.5, 0. ],
       [1. , 0.5, 0.5],
       [1. , 0.5, 1. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0.5],
       [0. , 1. , 1. ],
       [0.5, 1. , 0. ],
       [0.5, 1. , 0.5],
       [0.5, 1. , 1. ],
       [1. , 1. , 0. ],
       [1. , 1. , 0.5],
       [1. , 1. , 1. ]])

In [3]:
# 전체 노드의 수
num_nodes = nodes.shape[0]
num_nodes

27

In [4]:
# 요소 연결 (2x2x2 정육면체 기준)
elements = []
for i in range(num_elements_per_side):
    for j in range(num_elements_per_side):
        for k in range(num_elements_per_side):
            n0 = i * num_nodes_per_side**2 + j * num_nodes_per_side + k
            n1 = n0 + 1
            n2 = n0 + num_nodes_per_side
            n3 = n2 + 1
            n4 = n0 + num_nodes_per_side**2
            n5 = n4 + 1
            n6 = n4 + num_nodes_per_side
            n7 = n6 + 1
            elements.append([n0, n1, n3, n2, n4, n5, n7, n6])

elements = np.array(elements)
elements

array([[ 0,  1,  4,  3,  9, 10, 13, 12],
       [ 1,  2,  5,  4, 10, 11, 14, 13],
       [ 3,  4,  7,  6, 12, 13, 16, 15],
       [ 4,  5,  8,  7, 13, 14, 17, 16],
       [ 9, 10, 13, 12, 18, 19, 22, 21],
       [10, 11, 14, 13, 19, 20, 23, 22],
       [12, 13, 16, 15, 21, 22, 25, 24],
       [13, 14, 17, 16, 22, 23, 26, 25]])

In [5]:
# 물리적 특성
E = 210e9  # 탄성 계수 (Pa)
nu = 0.3   # 포아송 비율

# 유도된 요소 강성 행렬 계산 함수
def element_stiffness_matrix(E, nu, L):
    """ 3D 8-node hex element stiffness matrix """
    C = E / ((1 + nu) * (1 - 2 * nu)) * np.array([
        [1 - nu, nu, nu, 0, 0, 0],
        [nu, 1 - nu, nu, 0, 0, 0],
        [nu, nu, 1 - nu, 0, 0, 0],
        [0, 0, 0, (1 - 2 * nu) / 2, 0, 0],
        [0, 0, 0, 0, (1 - 2 * nu) / 2, 0],
        [0, 0, 0, 0, 0, (1 - 2 * nu) / 2],
    ])
    return C * L**3

# 시스템 강성 행렬 초기화
K = np.zeros((3 * num_nodes, 3 * num_nodes))

In [6]:
# 각 요소의 강성 행렬을 시스템 강성 행렬에 추가
L = length / num_elements_per_side
C = element_stiffness_matrix(E, nu, L)
C.shape

(6, 6)

In [7]:
def hex_element_stiffness(E, nu, L):
    """Simplified 3D element stiffness matrix for demonstration"""
    k = E / ((1 + nu) * (1 - 2 * nu)) * np.eye(6) * L**3
    Ke = np.zeros((24, 24))
    for i in range(8):
        for j in range(8):
            if i == j:
                Ke[3*i:3*i+3, 3*j:3*j+3] = k[:3, :3]
    return Ke

for el in elements:
    # 요소 강성 행렬
    Ke = hex_element_stiffness(E, nu, L)
    # 요소 노드 인덱스
    node_indices = np.repeat(3 * el, 3) + np.tile([0, 1, 2], len(el))
    for i, ni in enumerate(node_indices):
        for j, nj in enumerate(node_indices):
            K[ni, nj] += Ke[i, j]

# 경계 조건 (아래 면 고정)
fixed_dofs = np.where(nodes[:, 2] == 0)[0]
fixed_dofs = np.repeat(3 * fixed_dofs, 3) + np.tile([0, 1, 2], len(fixed_dofs))

# 하중 벡터 초기화 (위 면에 작용하는 외력)
F = np.zeros(3 * num_nodes)
top_dofs = np.where(nodes[:, 2] == length)[0]
F[3 * top_dofs + 2] = -1000  # Z 방향으로 -1000N의 힘

# 고정된 자유도 제거
free_dofs = np.setdiff1d(np.arange(3 * num_nodes), fixed_dofs)
K_reduced = K[np.ix_(free_dofs, free_dofs)]
F_reduced = F[free_dofs]

# 변위 계산
displacements = np.zeros(3 * num_nodes)
displacements[free_dofs] = np.linalg.solve(K_reduced, F_reduced)

# 결과 출력
print("Displacements at nodes:")
print(displacements.reshape(-1, 3))

Displacements at nodes:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.98095238e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -9.90476190e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.98095238e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -9.90476190e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.95238095e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -9.90476190e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00

In [8]:
# 변형 확대 비율 (시각화를 위해 실제 변형보다 크게 보이도록 설정)
scale_factor = 1e2

# 원래 위치에서의 노드 좌표
original_nodes = nodes.copy()

# 변형 후의 노드 좌표 (변위를 반영)
deformed_nodes = original_nodes + displacements.reshape(-1, 3) * scale_factor

# 각 요소의 면을 정의
faces = [[0, 1, 2, 3], [4, 5, 6, 7], [0, 1, 5, 4], [2, 3, 7, 6], [0, 2, 6, 4], [1, 3, 7, 5]]
faces

[[0, 1, 2, 3],
 [4, 5, 6, 7],
 [0, 1, 5, 4],
 [2, 3, 7, 6],
 [0, 2, 6, 4],
 [1, 3, 7, 5]]

In [9]:
# PyVista에서 사용할 정육면체의 원래 및 변형된 좌표
original_mesh = pv.PolyData()
deformed_mesh = pv.PolyData()

# # 요소를 원래 및 변형된 좌표에 추가
# for el in elements:
#     for face in faces:
#         original_mesh.faces = np.hstack([np.array([4] + [el[i] for i in face])])
#         deformed_mesh.faces = np.hstack([np.array([4] + [el[i] for i in face])])

original_mesh.points = original_nodes
deformed_mesh.points = deformed_nodes

In [10]:
original_mesh.points

pyvista_ndarray([[0. , 0. , 0. ],
                 [0. , 0. , 0.5],
                 [0. , 0. , 1. ],
                 [0.5, 0. , 0. ],
                 [0.5, 0. , 0.5],
                 [0.5, 0. , 1. ],
                 [1. , 0. , 0. ],
                 [1. , 0. , 0.5],
                 [1. , 0. , 1. ],
                 [0. , 0.5, 0. ],
                 [0. , 0.5, 0.5],
                 [0. , 0.5, 1. ],
                 [0.5, 0.5, 0. ],
                 [0.5, 0.5, 0.5],
                 [0.5, 0.5, 1. ],
                 [1. , 0.5, 0. ],
                 [1. , 0.5, 0.5],
                 [1. , 0.5, 1. ],
                 [0. , 1. , 0. ],
                 [0. , 1. , 0.5],
                 [0. , 1. , 1. ],
                 [0.5, 1. , 0. ],
                 [0.5, 1. , 0.5],
                 [0.5, 1. , 1. ],
                 [1. , 1. , 0. ],
                 [1. , 1. , 0.5],
                 [1. , 1. , 1. ]])

In [11]:
# 각 요소의 면을 정의
faces = []
for el in elements:
    faces.extend([
          [4, el[0], el[1], el[3], el[2]],
          # [4, el[4], el[5], el[7], el[6]],
          # [4, el[0], el[1], el[5], el[4]],
          # [4, el[2], el[3], el[7], el[6]],
          # [4, el[0], el[2], el[6], el[4]],
          # [4, el[1], el[3], el[7], el[5]]
         ])

faces = np.hstack(faces)
original_mesh.faces = faces
original_mesh.faces 

array([ 4,  0,  1,  3,  4,  4,  1,  2,  4,  5,  4,  3,  4,  6,  7,  4,  4,
        5,  7,  8,  4,  9, 10, 12, 13,  4, 10, 11, 13, 14,  4, 12, 13, 15,
       16,  4, 13, 14, 16, 17])

In [12]:
# import pyvista as pv
#pv.global_theme.trame.server_proxy_enabled = True
pv.set_jupyter_backend('client')

# PyVista 플로터 설정
plotter = pv.Plotter(shape=(1, 2), window_size=[1600, 800])
# plotter.subplot(0, 0)
plotter.add_mesh(original_mesh, show_edges=True, color='cyan', edge_color='r', opacity=0.5)
plotter.add_text("Original Position", position='upper_left', font_size=12)
# plotter.subplot(0, 1)
# plotter.add_mesh(deformed_mesh, show_edges=True, color='cyan', edge_color='r', opacity=0.5)
# plotter.add_text("Deformed Position", position='upper_left', font_size=12)
plotter.show()

Widget(value='<iframe src="http://localhost:33593/index.html?ui=P_0x7faa2db2fd60_0&reconnect=auto" class="pyvi…

In [13]:
# import pyvista as pv
# import numpy as np

# 정육면체의 꼭지점 좌표 정의
points = np.array([
    [0, 0, 0],  # 점 0
    [1, 0, 0],  # 점 1
    [1, 1, 0],  # 점 2
    [0, 1, 0],  # 점 3
    [0, 0, 1],  # 점 4
    [1, 0, 1],  # 점 5
    [1, 1, 1],  # 점 6
    [0, 1, 1]   # 점 7
])

# 정육면체의 면을 정의 (각 면은 4개의 점으로 구성)
faces = np.hstack([
    [4, 0, 1, 2, 3],  # 면 1
    [4, 4, 5, 6, 7],  # 면 2
    [4, 0, 1, 5, 4],  # 면 3
    [4, 1, 2, 6, 5],  # 면 4
    [4, 2, 3, 7, 6],  # 면 5
    [4, 3, 0, 4, 7]   # 면 6
])

# PolyData 객체 생성
cube = pv.PolyData(points, faces)

# 플로터 생성
plotter = pv.Plotter()
plotter.add_mesh(cube, color='white', show_edges=True)
plotter.show()


/home/webtk/workspace/sandbox/jupyter_sandbox/.venv/lib/python3.10/site-packages/pyvista/core/utilities/points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value='<iframe src="http://localhost:33593/index.html?ui=P_0x7faa2db2fe50_1&reconnect=auto" class="pyvi…